In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Bronze_to_Silver/utils

In [0]:
from pyspark.sql.functions import to_date

# Meta ads

df_meta = spark.read.format("delta").load(bronze_base + "Meta_ads").drop("_rescued_data")
df_meta = clean_common(df_meta).dropDuplicates(["ad_id", "date", "platform_name"])
date_format = "M/d/yyyy" 

df_meta = df_meta.select(
    col("ad_id"),
    col("campaign_id"),
    col("campaign_name"),
    col("platform_name"),
    to_date(col("campaign_start_date"), date_format).alias("campaign_start_date"),
    to_date(col("campaign_end_date"), date_format).alias("campaign_end_date"),
    to_date(col("date"), date_format).alias("date"),
    col("impressions").cast(IntegerType()).alias("impressions"),
    col("reach").cast(IntegerType()).alias("reach"),
    col("clicks").cast(IntegerType()).alias("clicks"),
    col("likes").cast(IntegerType()).alias("likes"),
    col("comments").cast(IntegerType()).alias("comments"),
    col("shares").cast(IntegerType()).alias("shares"),
    col("link_clicks").cast(IntegerType()).alias("link_clicks"),
    col("click_through_rate").cast(DoubleType()).alias("click_through_rate"),
    col("conversions").cast(IntegerType()).alias("conversions"),
    col("ad_spend").cast(DoubleType()).alias("ad_spend"),
    current_timestamp().alias("ingestion_timestamp"),
    lit(current_user).alias("modified_by")
)

df_meta = df_meta.dropna(how="any")
display(df_meta)
df_meta.write.mode("overwrite").format("delta").save(silver_base + "Meta_ads")